In [1]:
import numpy as np
from functools import partial
import sobol_seq

In [2]:
# import numpy as np
# import random

# # Define the objective function to be minimized
# def objective_function(x):
#     f1 = np.exp(x[0] - x[1]) - np.sin(x[0] + x[1])
#     f2 = (x[0] * x[1]) ** 2 - np.cos(x[0] + x[1])
#     f_list = [f1, f2]
#     denom = sum(np.abs(f) for f in f_list)
#     F = 1 / (1 + denom)
#     return -F

# # Genetic Algorithm parameters
# population_size = 100
# num_variables = 2
# lower_bound = -10
# upper_bound = 10
# mutation_rate = 0.1
# num_generations = 1000

# # Initialize population
# population = []
# for _ in range(population_size):
#     individual = [random.uniform(lower_bound, upper_bound) for _ in range(num_variables)]
#     population.append(individual)

# # Main loop
# for generation in range(num_generations):
#     # Evaluate fitness
#     fitness_values = [objective_function(individual) for individual in population]
    
#     # Selection (roulette wheel selection)
#     selection_probs = [1 / (fitness + 1) for fitness in fitness_values]
#     total_probs = sum(selection_probs)
#     selection_probs = [prob / total_probs for prob in selection_probs]
    
#     selected_indices = random.choices(range(population_size), weights=selection_probs, k=population_size)
#     selected_population = [population[i] for i in selected_indices]
    
#     # Crossover (single-point crossover)
#     offspring_population = []
#     for i in range(0, population_size, 2):
#         parent1 = selected_population[i]
#         parent2 = selected_population[i + 1]
#         crossover_point = random.randint(1, num_variables - 1)
#         offspring1 = parent1[:crossover_point] + parent2[crossover_point:]
#         offspring2 = parent2[:crossover_point] + parent1[crossover_point:]
#         offspring_population.extend([offspring1, offspring2])
    
#     # Mutation (random mutation)
#     for individual in offspring_population:
#         for j in range(num_variables):
#             if random.random() < mutation_rate:
#                 individual[j] = random.uniform(lower_bound, upper_bound)
    
#     # Replacement
#     population = offspring_population

# # Final evaluation
# fitness_values = [objective_function(individual) for individual in population]
# best_individual = population[fitness_values.index(min(fitness_values))]

# print("Best solution:", best_individual)
# print("Best fitness:", min(fitness_values))


**Problem 1 Pak Kun**

In [3]:
def objective_function(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    return np.array([f1,f2])

dim = 2
boundaries = np.array([(-10,10) for _ in range (dim)])
pop_size=100
max_gen=1000
mutation_rate=0.5
num_l=20
theta=1e-3
tau_d=0.4
s_max=100
print_gen=True
Hm = 50

seed = np.random.randint(0,100)
seed

67

# Objective Function

In [4]:
# def root_objective_function(x:np.ndarray):
#     res = 0
#     F_array = objective_function(x)
#     for f in F_array:
#         res +=(f)**2
#     return res

# def root_objective_function(x:np.ndarray):
#     res = 0
#     F_array = objective_function(x)
#     for f in F_array:
#         res +=abs(f)
#     return res

def root_objective_function(x:np.ndarray):
    F_array = objective_function(x)
    denom = 0
    for f in F_array:
        denom += np.abs(f)
    res = 1/(1+denom)
    return -res

In [5]:
"""GENERATE POINTS USING SOBOL SEQUENCE"""
def generate_points(dim: int,
                    npoint:int,
                    low=-10,
                    high=10,
                    sobol = True):
    if type(low) != type(high):
        raise TypeError('The type of "low" and "high" should be the same.')
    if type(low) == int:
        boundaries = [(low,high) for _ in range (dim)]
    elif type(low) == list or type(low) == np.ndarray:
        if len(low) != len(high):
            raise TypeError('The length of "low" and "high" should be the same.')
        else:
            boundaries = [(low[i],high[i]) for i in range (len(low))]

    if sobol == True:
        # Generate Sobol sequence points
        sobol_points = sobol_seq.i4_sobol_generate(dim, npoint)
        # Scale the Sobol points to fit within the specified boundaries
        scaled_points = []
        for i in range(dim):
            a, b = boundaries[i]
            scaled_dim = a + sobol_points[:, i] * (b - a)
            scaled_points.append(scaled_dim)
        # Transpose the scaled points to get points per dimension
        scaled_points = np.array(list(map(list, zip(*scaled_points))))
    
    else:
        scaled_points = np.zeros((npoint, dim))
        for i in range(dim):
            min_val, max_val = boundaries[i]
            scaled_points[:, i] = np.random.uniform(min_val, max_val, npoint)

    return scaled_points

# def initialize_population(pop_size, dimensions, bounds):
#     population = np.random.rand(pop_size, dimensions)
#     lower_bounds, upper_bounds = np.asarray(bounds).T
#     diff = np.fabs(lower_bounds - upper_bounds)
#     return lower_bounds + population * diff

In [6]:
"""Roullete wheel selection"""
def selection(population: np.ndarray,
              fitness: np.ndarray):
    population_size = population.shape[0]
    selection_probs = np.array([1 / (fit + 1) for fit in fitness]) # add one to avoid negative probability
    total_probs = sum(selection_probs)
    selection_probs = np.array([prob / total_probs for prob in selection_probs])
    selected_indices = np.random.choice(a=np.arange(population_size),size=population_size,p=selection_probs)
    selected_population = np.array([population[i] for i in selected_indices])
    return selected_population

"""Single point crossover"""
def crossover(parent1, parent2):
    dimension = len(parent1)
    crossover_point = np.random.randint(1, dimension)
    offspring1 = np.append(parent1[:crossover_point], parent2[crossover_point:])
    offspring2 = np.append(parent2[:crossover_point], parent1[crossover_point:])
    return offspring1, offspring2

"""One point mutation"""
def mutate(individual,mutation_rate, boundaries):
    for j in range(len(individual)):
        if np.random.random() < mutation_rate:
            individual[j] = np.random.uniform(boundaries[j][0], boundaries[j][1])
    return individual

# def mutate(population:np.ndarray, 
#            mutation_rate:float,
#            boundaries: np.ndarray):
#     dimension = population.shape[1]
#     for individual in population:
#         for j in range(dimension):
#             if np.random.random() < mutation_rate:
#                 individual[j] = np.random.uniform(boundaries[j][0], boundaries[j][1])
#     return population

def recombination(population: np.ndarray,mutation_rate, boundaries):
    offspring_population = []
    population_size,dimension = population.shape
    for i in range(0, population_size, 2):
        parent1 = population[i]
        parent2 = population[i + 1]
        offspring1, offspring2 = crossover(parent1=parent1,parent2=parent2)
        offspring1 = mutate(individual=offspring1,mutation_rate=mutation_rate, boundaries=boundaries)
        offspring2 = mutate(individual=offspring2,mutation_rate=mutation_rate, boundaries=boundaries)
        offspring_population.extend([offspring1, offspring2])
    offspring_population = np.array(offspring_population)
    return offspring_population

In [7]:
def GA(objective_function,
       population_size,
       boundaries,
       max_generation,
       mutation_rate,
       seed=0,
       print_stat = False):
    
    np.random.seed(seed)
    dim = boundaries.shape[1]
    population = generate_points(dim=dim,npoint=population_size,low=boundaries[:,0],high=boundaries[:,1],sobol=True)
    fitness = np.asarray([objective_function(ind) for ind in population])
    best_idx = np.argmin(fitness)
    best_individual = population[best_idx]

    for generation in range(max_generation):
        selected_population = selection(population=population,fitness=fitness)
        offspring_population = recombination(population=selected_population,
                                             mutation_rate=mutation_rate,
                                             boundaries=boundaries)
        population = offspring_population
        fitness = np.asarray([objective_function(ind) for ind in population])
        if print_stat == True:
            best_idx = np.argmin(fitness)
            best_individual = population[best_idx]
            best_fitness = fitness[best_idx]
            print(f"=========Generation {generation}=========")
            print(f"Best Individual: {best_individual}")
            print(f"Best Score: {best_fitness}\n")

    return best_individual, best_fitness

GA(root_objective_function,pop_size,boundaries,max_gen,mutation_rate, print_stat=True)

=========Generation 0=========
Best Individual: [-4.82631866  0.        ]
Best Score: -0.476375484867794

=========Generation 1=========
Best Individual: [0.36299628 0.42073212]
Best Score: -0.5200381168481

=========Generation 2=========
Best Individual: [-0.05217269  9.47637394]
Best Score: -0.44544560946194806

=========Generation 3=========
Best Individual: [-6.02177331  0.12685839]
Best Score: -0.5819181481195108

=========Generation 4=========
Best Individual: [-5.96067143  0.        ]
Best Score: -0.4419273671940394

=========Generation 5=========
Best Individual: [-0.91669463  0.8044561 ]
Best Score: -0.5744642078927438

=========Generation 6=========
Best Individual: [-0.91669463  1.40093756]
Best Score: -0.4692069016672354

=========Generation 7=========
Best Individual: [-6.91444445  0.15270939]
Best Score: -0.5922039599648804

=========Generation 8=========
Best Individual: [-6.90261169  0.15270939]
Best Score: -0.5992032750508788

=========Generation 9=========
Best Indivi

=========Generation 36=========
Best Individual: [-0.98817865  0.81132845]
Best Score: -0.5942017974181096

=========Generation 37=========
Best Individual: [0.64477633 0.64465667]
Best Score: -0.8738701214670836

=========Generation 38=========
Best Individual: [0.64477633 1.17028242]
Best Score: -0.4565632332082406

=========Generation 39=========
Best Individual: [-0.17828319  5.99106621]
Best Score: -0.586578138262709

=========Generation 40=========
Best Individual: [-5.99446371 -0.19295747]
Best Score: -0.6968265292785738

=========Generation 41=========
Best Individual: [-6.94179471 -0.01751872]
Best Score: -0.4180648420661203

=========Generation 42=========
Best Individual: [-0.28125826  0.81789331]
Best Score: -0.503877557751791

=========Generation 43=========
Best Individual: [-0.09712236  1.21416578]
Best Score: -0.48690904268988283

=========Generation 44=========
Best Individual: [-0.17656423  7.08539604]
Best Score: -0.4274591611977346

=========Generation 45=========
B

(array([0.16444329, 6.11889732]), -0.9853179857031704)

In [8]:
# population = generate_points(dim=dim,npoint=pop_size,low=boundaries[:,0],high=boundaries[:,1],sobol=True)
# fitness = np.asarray([root_objective_function(ind) for ind in population])
# best_idx = np.argmin(fitness)
# best = population[best_idx]

# for generation in range(max_gen):
#     selected_population = selection(population=population,fitness=fitness)
#     offspring_population = crossover(population=selected_population)
#     mutated_population = mutate(population=offspring_population,mutation_rate=mutation_rate,boundaries=boundaries)

#     population = mutated_population
#     fitness = np.asarray([root_objective_function(ind) for ind in population])
#     best_idx = np.argmin(fitness)
#     best_individual = population[best_idx]
#     best_fitness = fitness[best_idx]

# print("Best solution:", best_individual)
# print("Best fitness:", best_fitness)